# Generate a series of hkl0 values

Tricky things
* Make sure Ei is consistent with the beam simulation
* Make sure the angles are in correct units

In [1]:
pwd

u'/SNS/users/lj7/notebooks/.rk-work/jnrk-analysis-mcvine-unstable'

In [2]:
# some goodies
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt

In [37]:
import numpy as np, os
import histogram.hdf as hh, histogram as H

In [4]:
import mcvine.cli
from mcvine_workflow.DGS import ARCS
from mcvine_workflow import singlextal as sx
from mcvine_workflow.sample import loadSampleYml
from mcvine_workflow.singlextal import io as sxio, coords_transform, dynrange
from mcvine_workflow.singlextal.resolution import use_covmat, use_res_comps

In [5]:
workdir = '/SNS/users/lj7/simulations/ARCS/resolution-fit-covmat/3-generate-samples-for-fitting'
!mkdir -p {workdir}
%cd {workdir}

/SNS/users/lj7/simulations/ARCS/resolution-fit-covmat/3-generate-samples-for-fitting


# Inputs

## Instrument

ARCS

In [6]:
instrument = use_covmat.instrument(
    name = 'ARCS',
    detsys_radius = "3.*meter",
    L_m2s = "13.6*meter",
    L_m2fc = "11.61*meter",
    offset_sample2beam = "-0.15*meter" # offset from sample to saved beam
    )
pixel = use_covmat.pixel(
    radius = "0.5*inch",
    height = "meter/128",
    pressure = "10*atm",
    )

## Sample

In [7]:
%%file Si.yml
name: Si
chemical_formula: Si2
lattice: 
 constants: 5.490700041, 5.490700041, 5.490700041, 90, 90, 90
 basis_vectors:
  - 5.490700041, 0, 0
  - 0, 5.490700041, 0
  - 0, 0, 5.490700041
 primitive_basis_vectors:
  - 0.0, 2.71526503565, 2.71526503565
  - 2.71526503565, 0.0, 2.71526503565
  - 2.71526503565, 2.71526503565, 0.0
excitations:
  - type: deltafunction
    hkl: -16/3.,-8/3.,8/3.
    E: 40.
    dE: 0.5
orientation:
 u: -1, 1, -1
 v: 2, 1, -1
shape: hollowCylinder in_radius="5./8*inch" out_radius="1.*inch" height="1.5*inch"
temperature: 100*K

Writing Si.yml


In [8]:
sample_yml = './Si.yml'

## ** Dynamics parameters

In [19]:
# Ei = 100
q = 0
# E = 20.  # energy transfer

In [20]:
xo = sxio.loadXtalOriFromSampleYml('Si.yml')
# change psi if needed
# xo.psi = np.deg2rad(30.)
# print np.rad2deg(xo.psi)

## Compute hkl0

In [11]:
# Beam simulations by Ricky
!ls /SNS/users/p63/ORNL_public_research/MCViNE_Covmat_comparison/mcvine_resolution/beams

beam_1000      beam_125_1e9  beam_30	  beam_500_1e9
beam_1000_1e9  beam_250      beam_30_1e9  beam_60
beam_125       beam_250_1e9  beam_500	  beam_60_1e9


In [ ]:
Eis = [30, 60, 125, 250, 500, 1000]
NEis = len(Eis)

# Do One Ei first

In [14]:
np.random.random()

In [22]:
Ei = 1000

N_samples = 10
theta_min = -5
theta_max = 120
thetas = np.random.random(N_samples)*(theta_max-theta_min) + theta_min

z_min = -1
z_max = 1.
zs = np.random.random(N_samples)*(z_max-z_min) + z_min

E_min = -Ei*.95
E_max = Ei*.95
Es = np.random.random(N_samples)*(E_max-E_min) + E_min

rs = np.ones(N_samples)*3. # radius of the cylinder of the det system
print zs
print thetas
print Es

hkl0s = coords_transform.rtzE2hkl(r=rs, theta=np.deg2rad(thetas), z=zs, E=Es, xtalori=xo, Ei=Ei)
print hkl0s

[ 0.79645617 -0.12447938 -0.4510571  -0.90041401  0.20508299 -0.13451755
 -0.43967647 -0.36510042 -0.48784603 -0.29954128]
[  37.52674867  103.26817695   23.44515591    4.98238344   15.08630041
  103.33149288   83.63016973   12.8987237     7.00233949    4.63569268]
[-372.15937953  163.60481626  251.96909236  703.87109218 -913.69693183
   35.94861461  369.4388513  -361.86319412 -141.19699362  310.82373764]
[[-11.94155079  -0.19300258   8.35334387]
 [-27.34841118   5.92307836  -6.95242962]
 [ -7.7205586   -2.02583456  -1.46533636]
 [ -6.03808964   2.85026854  -7.09731339]
 [ -1.94439204  -5.22066611   7.78212714]
 [-28.55084298   5.51313513  -6.70719705]
 [-22.35665595   2.43472139  -5.5609156 ]
 [ -2.62128316  -5.37257233   1.54502605]
 [ -1.49889172  -3.8509597   -0.804145  ]
 [ -3.00430076   0.3149843   -2.55422957]]


In [24]:
hkl_dir = np.array([0,0,1]) # this could be randomized too
hkls = hkl0s+q*hkl_dir

In [33]:
# hklE array
print np.hstack((hkls, Es[:, np.newaxis]))

[[ -1.19415508e+01  -1.93002584e-01   8.35334387e+00  -3.72159380e+02]
 [ -2.73484112e+01   5.92307836e+00  -6.95242962e+00   1.63604816e+02]
 [ -7.72055860e+00  -2.02583456e+00  -1.46533636e+00   2.51969092e+02]
 [ -6.03808964e+00   2.85026854e+00  -7.09731339e+00   7.03871092e+02]
 [ -1.94439204e+00  -5.22066611e+00   7.78212714e+00  -9.13696932e+02]
 [ -2.85508430e+01   5.51313513e+00  -6.70719705e+00   3.59486146e+01]
 [ -2.23566560e+01   2.43472139e+00  -5.56091560e+00   3.69438851e+02]
 [ -2.62128316e+00  -5.37257233e+00   1.54502605e+00  -3.61863194e+02]
 [ -1.49889172e+00  -3.85095970e+00  -8.04145005e-01  -1.41196994e+02]
 [ -3.00430076e+00   3.14984299e-01  -2.55422957e+00   3.10823738e+02]]


** we have not randomize psi angles yet. could be done **

## ** Other parameters

In [35]:
# scan
class psi_scan:
    min = -5
    max = 90.
    step = 1.

# Setup simulations

In [41]:
!rm -rf tmp.use_res_comps/
!mkdir -p tmp.use_res_comps/

In [42]:
outdir = 'tmp.use_res_comps'
beam = '/SNS/users/p63/ORNL_public_research/MCViNE_Covmat_comparison/mcvine_resolution/beams/beam_1000_1e9/'
for E, hkl in zip(Es, hkls):
    print E,hkl
    outdir1 = os.path.join(outdir, 'E%s_hkl%s' % (E, '%s,%s,%s' % tuple(hkl)))
    use_res_comps.setup(outdir1, sample_yml, beam, E, hkl, hkl_dir, psi_scan, instrument, pixel)

-372.159379526 [-11.94155079  -0.19300258   8.35334387]
Ei=1010.94552405, t0=5.03711447572
Computed:
* psi=-0.0460465757048 degree
* Q=[  2.23096913 -15.15191814  -6.60306351]
* hkl2Qmat=[[ -6.59929439e-01   9.34874059e-01   7.77555548e-17]
 [  6.61055787e-01   4.66640582e-01  -8.09165116e-01]
 [ -6.61055787e-01  -4.66640582e-01  -8.09165116e-01]]
* ki=22.0879917858
* vectors ki=[ 22.08799179   0.           0.        ], kf=[ 19.85702265  15.15191814   6.60306351]
These two numbers should be very close:
  -372.159379526
  -372.159379526
  Ei=1010.94552405, Ef=1383.10490358
* Ef=1383.10490358
* pixel positon=[ 2.38497685  1.81985863  0.79307728]
* pixel_position=[ 2.38497685  1.81985863  0.79307728]
t_m2s=0.000982956131398, t_s2p=0.000190761290625, t_m2p=0.00117371742202
163.604816257 [-27.34841118   5.92307836  -6.95242962]
Ei=1010.94552405, t0=5.03711447572
Computed:
* psi=-0.420942517355 degree
* Q=[ 26.43090469 -19.73244867   0.83291513]
* hkl2Qmat=[[-0.65379832  0.93917205  0.      

In [43]:
ls tmp.use_res_comps/

E-141.196993625_hkl-1.49889172417,-3.85095970255,-0.804145004825/
E163.604816257_hkl-27.348411181,5.92307836282,-6.9524296165/
E251.969092361_hkl-7.7205586027,-2.02583456007,-1.46533635956/
E310.823737639_hkl-3.00430075938,0.314984298655,-2.55422956784/
E35.9486146145_hkl-28.5508429833,5.51313513282,-6.70719704811/
E-361.863194118_hkl-2.62128315547,-5.37257232899,1.54502605159/
E369.438851298_hkl-22.3566559529,2.43472138551,-5.56091560488/
E-372.159379526_hkl-11.9415507873,-0.193002583752,8.35334387063/
E703.871092184_hkl-6.03808964002,2.85026854386,-7.09731339035/
E-913.696931831_hkl-1.94439203643,-5.22066611166,7.78212714387/
